In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

import shared_utils

from siuba import *
import pandas as pd
import geopandas as gpd
import numpy as np

pd.set_option('display.max_columns', None) 

import datetime as dt
import time

from calitp import get_engine
from calitp.tables import tbls

engine = get_engine()
connection = engine.connect()

GCS_FILE_PATH = 'gs://calitp-analytics-data/data-analyses/ahsc_grant/'

import gcsfs
fs = gcsfs.GCSFileSystem()

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


# Collating Big Stop Table

In [2]:
# read in and concatenate all geoparquets
fs_list = fs.ls(f"{GCS_FILE_PATH}tool_data/")

#fs_list[1:]

In [3]:
filelist = []
for f in fs_list[1:]:
    try: 
        test_gpqt = gpd.read_parquet(f"gs://{f}")
        filelist.append(f)
    except:
        print(f"error on {f.split('tool_data/')}")
        
#filelist

error on ['calitp-analytics-data/data-analyses/ahsc_grant/', 'trips_perstop_116.parquet']
error on ['calitp-analytics-data/data-analyses/ahsc_grant/', 'trips_perstop_118.parquet']
error on ['calitp-analytics-data/data-analyses/ahsc_grant/', 'trips_perstop_186.parquet']
error on ['calitp-analytics-data/data-analyses/ahsc_grant/', 'trips_perstop_199.parquet']
error on ['calitp-analytics-data/data-analyses/ahsc_grant/', 'trips_perstop_21.parquet']
error on ['calitp-analytics-data/data-analyses/ahsc_grant/', 'trips_perstop_212.parquet']
error on ['calitp-analytics-data/data-analyses/ahsc_grant/', 'trips_perstop_213.parquet']
error on ['calitp-analytics-data/data-analyses/ahsc_grant/', 'trips_perstop_216.parquet']
error on ['calitp-analytics-data/data-analyses/ahsc_grant/', 'trips_perstop_23.parquet']
error on ['calitp-analytics-data/data-analyses/ahsc_grant/', 'trips_perstop_254.parquet']
error on ['calitp-analytics-data/data-analyses/ahsc_grant/', 'trips_perstop_277.parquet']
error on ['c

In [34]:
gdf_wkd = (pd.concat(
        gpd.read_parquet(f"gs://{f}")
        for f in filelist
        )
           >> filter(_.daytype=="Weekday")
          )

In [35]:
gdf_wkd.head()

,calitp_itp_id,stop_name,stop_id,geometry,n_trips,n_routes,daytype,analysis_date,sum_tracts,sum_total_pop,sum_households,sum_not_us_citizen_pop,sum_youth_pop,sum_seniors_pop,sum_pop_determined_poverty_status,sum_poverty,sum_no_car,sum_no_cars,sum_land_area,sum_jobs,land_area_sqkm,pop_density,job_density,pct_not_us_citizen_pop,pct_youth_pop,pct_seniors_pop,pct_poverty,pct_pop_workers_no_car,pct_hh_no_cars
0,10,Stockton Station,3400001,POINT (-121.27895 37.95706),8,0,Weekday,2022-11-02,3,16656.00,6416.00,4894.00,6072.00,1520.00,16456.00,7588.00,804.00,2460.00,7572848.00,36500.00,7.57,2199.44,4819.85,29.38,36.46,9.13,46.11,4.83,38.34
1,10,Stockton Station,3400001,POINT (-121.27895 37.95706),8,0,Weekday,2022-11-02,3,16656.00,6416.00,4894.00,6072.00,1520.00,16456.00,7588.00,804.00,2460.00,7572848.00,36500.00,7.57,2199.44,4819.85,29.38,36.46,9.13,46.11,4.83,38.34
2,10,Lathrop/Manteca Station,3400002,POINT (-121.26366 37.79791),8,0,Weekday,2022-11-02,2,30612.00,8320.00,4548.00,11340.00,2630.00,30588.00,2742.00,54.00,92.00,94456838.00,24000.00,94.46,324.08,254.08,14.86,37.04,8.59,8.96,0.18,1.11
3,10,Lathrop/Manteca Station,3400002,POINT (-121.26366 37.79791),8,0,Weekday,2022-11-02,2,30612.00,8320.00,4548.00,11340.00,2630.00,30588.00,2742.00,54.00,92.00,94456838.00,24000.00,94.46,324.08,254.08,14.86,37.04,8.59,8.96,0.18,1.11
4,10,Tracy Station,3400003,POINT (-121.43387 37.69647),8,0,Weekday,2022-11-02,3,42968.00,11960.00,3100.00,17280.00,2986.00,42884.00,2024.00,340.00,194.00,800367410.00,6538.00,800.37,53.69,8.17,7.21,40.22,6.95,4.72,0.79,1.62


In [36]:
gdf_wkd=gdf_wkd.drop_duplicates(ignore_index=True)

In [37]:
gdf_wkd.n_routes.value_counts()

#~2000 stops with no associated routes; missing route_id?

1     65287
2     20260
3      5352
4      3703
0      1860
6      1371
5       719
8       529
7       213
9       184
10      172
12       46
14       43
13       14
11       13
18        8
20        6
16        4
24        3
17        2
19        2
23        2
15        2
Name: n_routes, dtype: int64

In [38]:
gdf_wkd.n_trips.describe()

count   99795.00
mean       43.42
std        71.40
min         1.00
25%        15.00
50%        28.00
75%        57.00
max     12140.00
Name: n_trips, dtype: float64

Note: Running into memory issues adding spatial weights matrix. Proceeding without spatially-lagged factors for now.

In [7]:
# put coefficients into arrays - variable order from spatial_regression_exploration_kmk
# in future, save out coefficients somewhere 
import numpy as np

wkd_coeff = np.array([-0.1610594,0.0001214,-0.0000173,0.0224169,-0.0152673,-0.0505976,-0.0423512,0.0111763])
sat_coeff = np.array([-0.1424400,0.0001344,-0.0000186,0.0256008,-0.0169793,-0.0408743,-0.0419725,0.0126354])
sun_coeff = np.array([-0.1082477,0.0001477,-0.0000202,0.0209053,-0.0145447,-0.0449611,-0.0502937, 0.0132250])

In [39]:
# multiply by coefficients - everything except n_trips
gdf_wkd.loc[:, ['n_routes', 'pop_density', 'job_density','pct_not_us_citizen_pop',
                    'pct_youth_pop', 'pct_seniors_pop', 'pct_pop_workers_no_car', 'pct_poverty']] *= wkd_coeff

In [40]:
# create stop-specific ridership factor
gdf_wkd = (gdf_wkd
           >> mutate(control_vars_sum = _.n_routes+_.pop_density+_.job_density+_.pct_not_us_citizen_pop+_.pct_youth_pop+_.pct_seniors_pop+_.pct_pop_workers_no_car+_.pct_poverty,
                     control_vars_factor = _.control_vars_sum+(_.n_trips*0.0200865)+6.8721538
           )
          )

In [41]:
# baseline ridership model estimate: np.exp(control_vars_factor + (n_trips*n_trips_coeff) + correction factor)
gdf_wkd['model_est_ridership'] = np.exp(gdf_wkd['control_vars_factor']+(2.434/2))

In [42]:
gdf_wkd.model_est_ridership.value_counts()

2499.95    69
1826.54    68
3220.45    48
872.80     44
1333.40    43
           ..
2681.34     1
1994.60     1
870.98      1
1545.50     1
655.30      1
Name: model_est_ridership, Length: 46795, dtype: int64

In [43]:
pd.options.display.float_format = '{:.2f}'.format

In [44]:
gdf_wkd.model_est_ridership.describe()

count                                             98798.00
mean    24660797922224379315830516709300495823294144626...
std     77514187837725719088352234700829067272642367498...
min                                                  43.79
25%                                                1711.57
50%                                                2968.43
75%                                                6400.29
max     24364375131199240854921409288285679099102503136...
Name: model_est_ridership, dtype: float64

In [55]:
orgnames = (tbls.airtable.california_transit_organizations() 
    >> select(_.itp_id, _.name, _.caltrans_district)
    >> collect()
           )

In [51]:
outliers = pd.DataFrame(gdf_wkd
            >> filter(_.model_est_ridership>10000000)
                       )

In [56]:
outliers >> count(_.calitp_itp_id) >> left_join(_,orgnames, {"calitp_itp_id":"itp_id"})

,calitp_itp_id,n,itp_id,name,caltrans_district
0,14,11,14.00,Anaheim Transportation Network,12 - Irvine
1,112,3,112.00,Foothill Transit,07 - Los Angeles
2,182,30,182.00,Los Angeles County Metropolitan Transportation...,07 - Los Angeles
3,183,1,183.00,City of Los Angeles,07 - Los Angeles
4,279,12,279.00,San Francisco Bay Area Rapid Transit District,04 - Oakland
5,281,2,281.00,San Francisco International Airport,04 - Oakland
6,282,24,282.00,City and County of San Francisco,04 - Oakland
7,293,1,293.00,Santa Barbara Metropolitan Transit District,05 - San Luis Obispo
8,294,4,294.00,Santa Clara Valley Transportation Authority,04 - Oakland
9,484,1,484.00,Stanislaus Regional Transit Authority,10 - Stockton


In [ ]:
# explode routes